Stats data from https://datafinder.stats.govt.nz/layer/104430-meshblock-2020-centroid-inside/

In [25]:
import pandas as pd
import shapely
from shapely import Polygon
from shapely import wkt
import dash_leaflet as dl

locales_df = pd.read_csv("data\\nz_suburbs_and_localities.csv")
ece_services_df = pd.read_csv("data\\ece_services.csv")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
#drop services with bad latitude or longitude data
ece_services_df = ece_services_df.drop(ece_services_df[ece_services_df['Latitude'] > 90].index)
ece_services_df = ece_services_df[ece_services_df['Latitude'].notna()]
ece_services_df = ece_services_df[ece_services_df['Longitude'].notna()]

#filter ece services to just columns needed
ece_services_df = ece_services_df[['Service Name', 'Latitude', 'Longitude', 'Town / City', 'General Electorate', 'Street', 'Suburb', 'Town / City', 'Territorial Authority', 'Service Type', '20 Hours ECE', 'Equity Index', 'Max. Licenced Positions', 'Under 2\'s']]

In [ ]:
#tidy up territorial authorities
ece_services_df['Territorial Authority'].replace(['Auckland.*'], 'Auckland', regex=True, inplace=True)

In [ ]:
from shapely.geometry import Point, Polygon
import shapely.wkt

def ece_capacity(locale_pop, locale_ta, locale_poly):
    ece_capacity = 0
    ece_services = []

    if (pd.isna(locale_pop)):
        return(0)
    else:

        #get services just for this ta
        ece_services_ta_df = ece_services_df[ece_services_df.apply(lambda x: x['Territorial Authority'] in locale_ta, axis=1)]

        polygon = shapely.wkt.loads(locale_poly)


        # if (len(ece_services_ta_df) == 0):
        #     print(locale_ta + " returned " + str(len(ece_services_ta_df)) + " services.")

        for item, row in ece_services_ta_df.iterrows():

            point = Point(row['Longitude'], row['Latitude'])

            if (point.within(polygon)):
                ece_capacity = ece_capacity + row['Max. Licenced Positions']
                ece_services.append({'name': row['Service Name'], 'lat' : row['Latitude'], 'lng' : row['Longitude'], 'max' : row['Max. Licenced Positions']})

        return([ece_capacity, ece_services])

In [ ]:
def ece_demand(population, ece_profile):
    prop_U5 = 0.059
    prop_att = 0.607
    

    if (pd.isna(population)):
        return(0)
    else:

        U5_pop = round(population * prop_U5)
        ece_pop = round(U5_pop * prop_att)

        if (ece_profile[0] == 0):
            ece_capacity = 0
        else:
            ece_capacity = round(ece_pop/ece_profile[0], 2)

        result = {
            'U5_pop' : U5_pop,
            'ece_pop' : ece_pop,
            'ece_places' : ece_profile[0],
            'ece_services' : ece_profile[1],
            'ece_capacity' : ece_capacity        
        }

    return (result)

In [ ]:
#identifies if the ta is covered by the locale
def identify_ta(locale_ta, target_tas):
    
    for ta in target_tas:
        if ta in locale_ta:
            return (True)
    
    return (False)

In [ ]:
locales_df["ece_capacity"] = locales_df.apply(lambda row: ece_capacity(row['population_estimate'], row['territorial_authority_ascii'],row['WKT']), axis =1)

In [ ]:
locales_df["ece_demand"] = locales_df.apply(lambda row: ece_demand(row['population_estimate'], row['ece_capacity']), axis =1)

In [ ]:
locales_df['include'] = locales_df.apply(lambda row: identify_ta(row['territorial_authority_ascii'], ['Christchurch City', 'Selwyn District', 'Waimakariri']), axis=1)

In [ ]:
locales_df.drop(['ece_capacity'], inplace=True, axis=1)

In [ ]:
locales_df = pd.concat([locales_df.drop('ece_demand', axis=1),  pd.json_normalize(locales_df['ece_demand'])], axis=1)

In [ ]:
locales_df = locales_df[['WKT', 'id', 'name', 'U5_pop', 'ece_pop', 'ece_places', 'ece_services', 'ece_capacity', 'include']]

In [ ]:
def get_poly_coordinates(poly):

    #create geometry view
    polygon =  wkt.loads(poly)
    polygon = shapely.geometry.mapping(polygon)

    positions = polygon['coordinates']
    positions = [list(elem) for elem in list(positions[0])]

    for item in positions:
        item.reverse()

    return (positions)

In [ ]:
locales_df['positions'] = locales_df.apply(lambda row: get_poly_coordinates(row['WKT']), axis =1)

In [ ]:
locales_df

,WKT,id,name,U5_pop,ece_pop,ece_places,ece_services,ece_capacity,christchurch_city,positions
0,"POLYGON ((171.355078005649 -42.070367999046,17...",7140,Meybille Bay,NaN,NaN,NaN,NaN,NaN,False,"[[-42.070367999046, 171.355078005649], [-42.07..."
1,"POLYGON ((172.879629996903 -43.8992019989129,1...",7828,Whakamoa Bay,NaN,NaN,NaN,NaN,NaN,False,"[[-43.8992019989129, 172.879629996903], [-43.8..."
2,"POLYGON ((174.985845000394 -36.8789809991999,1...",7545,Sunkist Bay,NaN,NaN,NaN,NaN,NaN,False,"[[-36.8789809991999, 174.985845000394], [-36.8..."
3,"POLYGON ((174.882553784236 -37.7972490227562,1...",7072,Lorenzen Bay,NaN,NaN,NaN,NaN,NaN,False,"[[-37.7972490227562, 174.882553784236], [-37.7..."
4,"POLYGON ((175.086644006051 -36.8384020053753,1...",8442,Deadmans Bay,NaN,NaN,NaN,NaN,NaN,False,"[[-36.8384020053753, 175.086644006051], [-36.8..."
...,...,...,...,...,...,...,...,...,...,...
6551,"POLYGON ((168.351876007319 -46.4009030017748,1...",70,Avenal,79.0,48.0,392.0,"[{'name': 'Active Explorers Invercargill', 'la...",0.12,False,"[[-46.4009030017748, 168.351876007319], [-46.4..."
6552,"POLYGON ((170.503520992814 -45.8631389967932,1...",8799,North Dunedin,587.0,356.0,247.0,"[{'name': 'Kelsey-Yaralla Kindergarten', 'lat'...",1.44,False,"[[-45.8631389967932, 170.503520992814], [-45.8..."
6553,"POLYGON ((171.246843006687 -44.384725000598,17...",8764,Maori Hill,68.0,41.0,0.0,[],0.00,False,"[[-44.384725000598, 171.246843006687], [-44.38..."
6554,"POLYGON ((174.817220235269 -41.3437231286051,1...",942,Moa Point,2.0,1.0,0.0,[],0.00,False,"[[-41.3437231286051, 174.817220235269], [-41.3..."


In [ ]:
import pandas as pd
import geopandas as gpd

# Load your DataFrame (replace 'your_data.csv' with your file path)
df = pd.read_csv("data\\ece_demand.csv")

# Assuming you have a column named 'wkt_polygon' containing WKT data
# Convert the 'wkt_polygon' column to a GeoSeries
df['geometry'] = gpd.GeoSeries.from_wkt(df['WKT'])


In [ ]:
# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [ ]:
gdf = gdf[gdf['include'] == True]

In [ ]:
gdf = gdf[['name', 'ece_services', 'ece_capacity', 'geometry']]

In [ ]:
# Save the GeoDataFrame to a GeoJSON file (replace 'output.geojson' with your desired file path)
gdf.to_file('output.geojson', driver='GeoJSON')

print("GeoJSON file saved successfully.")

GeoJSON file saved successfully.


In [ ]:
import pandas as pd

regions_df = pd.read_csv("data\\ece_demand.csv")

In [ ]:
import pandas as pd
import shapely
from shapely.geometry import Point, Polygon
from shapely import wkt
import geopandas as gpd
import pyproj

from shapely.geometry import Point
from shapely.ops import transform

locales_df = pd.read_csv("data\\nz_suburbs_and_localities.csv")
ta_df = pd.read_csv("data\\ta.csv")

In [ ]:
def convert_coordinates(nztm_poly):

    nztm_poly = wkt.loads(nztm_poly)

    nztm = pyproj.CRS('EPSG:2193')
    nzgd2000 = pyproj.CRS('EPSG:4167')
    
    project = pyproj.Transformer.from_crs(nztm, nzgd2000, always_xy=True).transform
    nzgd2000_poly = transform(project, nztm_poly)

    return (nzgd2000_poly)

In [ ]:
ta_df['WKT_U'] = ta_df.apply(lambda row: convert_coordinates(row['WKT']), axis =1)

In [ ]:
def get_ta(poly):
    ta = ''
    polygon = wkt.loads(poly)

    for item, row in ta_df.iterrows():
        region_poly = row['WKT_U']

        if polygon.intersects(region_poly):
            ta= row['TA2022_V1_00_NAME_ASCII']
            
            return ta
        
    return ta

In [ ]:
#identify regions the locale polygon falls within
locales_df["ta"] = locales_df.apply(lambda row: get_ta(row['WKT']), axis =1)

In [ ]:
set(list(locales_df['ta']))

{'Area Outside Territorial Authority',
 'Ashburton District',
 'Auckland',
 'Buller District',
 'Carterton District',
 "Central Hawke's Bay District",
 'Central Otago District',
 'Christchurch City',
 'Clutha District',
 'Dunedin City',
 'Far North District',
 'Gisborne District',
 'Gore District',
 'Grey District',
 'Hamilton City',
 'Hastings District',
 'Hauraki District',
 'Horowhenua District',
 'Hurunui District',
 'Invercargill City',
 'Kaikoura District',
 'Kaipara District',
 'Kapiti Coast District',
 'Lower Hutt City',
 'Mackenzie District',
 'Manawatu District',
 'Marlborough District',
 'Masterton District',
 'Matamata-Piako District',
 'Napier City',
 'Nelson City',
 'New Plymouth District',
 'Opotiki District',
 'Otorohanga District',
 'Palmerston North City',
 'Porirua City',
 'Queenstown-Lakes District',
 'Rangitikei District',
 'Rotorua District',
 'Ruapehu District',
 'Selwyn District',
 'South Taranaki District',
 'South Waikato District',
 'South Wairarapa District'

# Create centroid for territorial authorities

In [ ]:
import pandas as pd
import shapely
from shapely import Polygon
from shapely import Point
from shapely import wkt
from shapely import wkt
import geopandas as gpd
import pyproj
from shapely.ops import transform


ta_df = pd.read_csv("data\\ta.csv")

ta_df['polygon'] = wkt.loads(ta_df['WKT'])
ta_df['centroid'] = shapely.centroid(ta_df['polygon'])


def convert_point(input_point):

    nztm = pyproj.CRS('EPSG:2193')
    nzgd2000 = pyproj.CRS('EPSG:4167')
        
    project = pyproj.Transformer.from_crs(nztm, nzgd2000, always_xy=True).transform
    nzgd2000_point = transform(project, input_point)

    return nzgd2000_point

ta_df['centroid_ll'] = ta_df.apply(lambda row: convert_point(row['centroid']), axis=1)

In [ ]:
ece_services_df['geometry'] = ece_services_df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

In [ ]:
gdf = gpd.GeoDataFrame(ece_services_df, geometry='geometry')

In [ ]:
gdf.to_file('services.geojson', driver='GeoJSON')

# Create TA summary

In [24]:
import json

with open('locales.geojson') as fp:
    data = json.load(fp)

In [41]:
tas = [
 'Area outside Territorial Authority',
 'Ashburton District',
 'Auckland',
 'Buller District',
 'Carterton District',
 "Central Hawke's Bay District",
 'Central Otago District',
 'Christchurch City',
 'Clutha District',
 'Dunedin City',
 'Far North District',
 'Gisborne District',
 'Gore District',
 'Grey District',
 'Hamilton City',
 'Hastings District',
 'Hauraki District',
 'Horowhenua District',
 'Hurunui District',
 'Invercargill City',
 'Kaikoura District',
 'Kaipara District',
 'Kawerau District',
 'Kapiti Coast District',
 'Lower Hutt City',
 'Mackenzie District',
 'Manawatu District',
 'Marlborough District',
 'Masterton District',
 'Matamata-Piako District',
 'Napier City',
 'Nelson City',
 'New Plymouth District',
 'Opotiki District',
 'Otorohanga District',
 'Palmerston North City',
 'Porirua City',
 'Queenstown-Lakes District',
 'Rangitikei District',
 'Rotorua District',
 'Ruapehu District',
 'Selwyn District',
 'South Taranaki District',
 'South Waikato District',
 'South Wairarapa District',
 'Southland District',
 'Stratford District',
 'Tararua District',
 'Tasman District',
 'Taupo District',
 'Tauranga City',
 'Thames-Coromandel District',
 'Timaru District',
 'Upper Hutt City',
 'Waikato District',
 'Waimakariri District',
 'Waimate District',
 'Waipa District',
 'Wairoa District',
 'Waitaki District',
 'Waitomo District',
 'Wellington City',
 'Western Bay of Plenty District',
 'Westland District',
 'Whakatane District',
 'Whanganui District',
 'Whangarei District']

In [42]:
#create a dictionary where each key is a ta and each value is a dictionary that contains stats for the specific ta
result = {k: {
    'total_ece_pop' : 0,
    'total_ece_places' : 0,
    'overall_ece_demand' : 0,
    'total_locales' : 0,
    'over_demand_locales' : 0,
    'near_demand_localtes': 0,
    'under_demand_locales' : 0,
    'no_population_locales' : 0,
    'total_services' : 0,
    'lat' : 0,
    'lng' : 0
} for k in tas}

In [44]:
for locale in data['features']:
    ta = locale['properties']['ta']

    result[ta]['total_ece_pop'] = result[ta]['total_ece_pop'] + locale['properties']['ece_pop']

    places = 0

    #check if there are services in the specific locale
    if (len(locale['properties']['ece_services']) != 0):
        
        #get the list of services for the locale
        services = locale['properties']['ece_services']

        #the the total services for the locale to the overall total for the ta
        result[ta]['total_services'] = result[ta]['total_services'] + len(services)
 
        #iterate through serives and add to the total number of places for the locale
        for service in services:
            places = places + service['max']

    #add the total places for the locale to the total places for the ta
    result[ta]['total_ece_places'] = result[ta]['total_ece_places'] + places

    #increment the total localities in the TA by 1
    result[ta]['total_locales'] = result[ta]['total_locales'] + 1

    #increment count of ece demand category
    if (locale['properties']['ece_capacity'] > 1):
        result[ta]['over_demand_locales'] = result[ta]['over_demand_locales'] + 1
    elif (locale['properties']['ece_capacity'] == -1):
        result[ta]['no_population_locales'] = result[ta]['no_population_locales'] + 1
    elif (0.9 <= locale['properties']['ece_capacity'] <= 1):
        result[ta]['near_demand_localtes'] = result[ta]['near_demand_localtes'] + 1
    else:
        result[ta]['under_demand_locales'] = result[ta]['under_demand_locales'] + 1


In [53]:
for ta in result:
    if(result[ta]['total_ece_places'] != 0):
        result[ta]['overall_ece_demand'] = result[ta]['total_ece_pop']/result[ta]['total_ece_places']
    else:
        result[ta]['overall_ece_demand'] = -1

In [54]:
result

{'Area outside Territorial Authority': {'total_ece_pop': 1.44,
  'total_ece_places': 0,
  'overall_ece_demand': -1,
  'total_locales': 440,
  'over_demand_locales': 2,
  'near_demand_localtes': 0,
  'under_demand_locales': 0,
  'no_population_locales': 438,
  'total_services': 0,
  'lat': 0,
  'lng': 0},
 'Ashburton District': {'total_ece_pop': 2643.9,
  'total_ece_places': 2382.0,
  'overall_ece_demand': 1.1099496221662468,
  'total_locales': 104,
  'over_demand_locales': 80,
  'near_demand_localtes': 4,
  'under_demand_locales': 20,
  'no_population_locales': 0,
  'total_services': 56,
  'lat': 0,
  'lng': 0},
 'Auckland': {'total_ece_pop': 122020.9599999999,
  'total_ece_places': 158288.0,
  'overall_ece_demand': 0.7708794096836141,
  'total_locales': 840,
  'over_demand_locales': 334,
  'near_demand_localtes': 28,
  'under_demand_locales': 218,
  'no_population_locales': 260,
  'total_services': 2896,
  'lat': 0,
  'lng': 0},
 'Buller District': {'total_ece_pop': 716.7800000000001,

In [44]:
with open('ta_summary.json', 'w') as fp:
    json.dump(result, fp)

In [1]:
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash import Dash, html, dcc, Output, Input
from dash_extensions.javascript import assign

# A few cities in Denmark.
cities = [dict(name="Aalborg", lat=57.0268172, lon=9.837735),
          dict(name="Aarhus", lat=56.1780842, lon=10.1119354),
          dict(name="Copenhagen", lat=55.6712474, lon=12.5237848)]
# Create drop down options.
dd_options = [dict(value=c["name"], label=c["name"]) for c in cities]
dd_defaults = [o["value"] for o in dd_options]
# Generate geojson with a marker for each city and name as tooltip.
geojson = dlx.dicts_to_geojson([{**c, **dict(tooltip=c['name'])} for c in cities])

In [2]:
geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [9.837735, 57.0268172]},
   'properties': {'name': 'Aalborg', 'tooltip': 'Aalborg'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [10.1119354, 56.1780842]},
   'properties': {'name': 'Aarhus', 'tooltip': 'Aarhus'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [12.5237848, 55.6712474]},
   'properties': {'name': 'Copenhagen', 'tooltip': 'Copenhagen'}}]}

In [ ]:
with open('ta_summary.json', 'w') as fp:
    json.dump(result, fp)

In [3]:
import json

with open('ta_summary.json') as fp:
    data = json.load(fp)

In [4]:
data

{'Area outside Territorial Authority': {'total_ece_pop': 1.0,
  'total_ece_places': 0,
  'overall_ece_demand': -1,
  'total_locales': 220,
  'over_demand_locales': 1,
  'near_demand_localtes': 0,
  'under_demand_locales': 0,
  'no_population_locales': 219,
  'total_services': 0,
  'lat': 0,
  'lng': 0},
 'Ashburton District': {'total_ece_pop': 1322.0,
  'total_ece_places': 1191.0,
  'overall_ece_demand': 1.11,
  'total_locales': 52,
  'over_demand_locales': 40,
  'near_demand_localtes': 2,
  'under_demand_locales': 10,
  'no_population_locales': 0,
  'total_services': 28,
  'lat': 0,
  'lng': 0},
 'Auckland': {'total_ece_pop': 61010.0,
  'total_ece_places': 79144.0,
  'overall_ece_demand': 0.77,
  'total_locales': 420,
  'over_demand_locales': 167,
  'near_demand_localtes': 14,
  'under_demand_locales': 109,
  'no_population_locales': 130,
  'total_services': 1448,
  'lat': 0,
  'lng': 0},
 'Buller District': {'total_ece_pop': 358.0,
  'total_ece_places': 272.0,
  'overall_ece_demand':